<a href="https://colab.research.google.com/github/weprintmoney/LLMPractice/blob/main/9.02%20Canadian%20Law%20LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Custom RAG Implementation</center></h1>
<h2><center>The Consolidated Acts and Regulations of Canada</center></h2>
<h3><center>Charlcye Mitchell & Matt Moore, May 2024</center></h3>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/semantic-search.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/semantic-search.ipynb)

# Objective
The goal of this assignment is to explore advanced applications of large language models in the legal domain. We will implement the LLaMA3 GGUF model with the Retrieval-Augmented Generation (RAG) technique, using a dataset consisting of the consolidated acts and regulations of Canada. This implementation aims to leverage the rich contextual understanding of the LLaMA3 model with the retrieval capabilities of RAG to enhance the accuracy and relevance of generated responses in legal contexts.

### Background

*   The model we will be downloading from Hugging Face is a **5-bit quantized version of the Llama 3 8B chat model**, made available by NousResearch. The model is made available in the **GGUF format** - a new format introduced by the Llama CPP team and a replacement for the earlier GGML format, with advantages such as better tokenization and support for special tokens. Llama is short for **L**arge **LA**nguage Model **M**eta **A**I. https://huggingface.co/NousResearch/Meta-Llama-3-8B-Instruct-GGUF

*   RAG (Retrieval Augmented Generation) combines a powerful transformer-based language model with a retrieval system, allowing the model to pull in relevant external information during the generation process. This combination is particularly potent for domains like law where precedent and specific details are crucial. https://www.llamaindex.ai/blog/a-cheat-sheet-and-some-recipes-for-building-advanced-rag-803a9d94c41b

*   FAISS (Facebook AI Similarity Search) empowers us with its state-of-the-art similarity search capabilities, allowing us to effortlessly find documents that closely match a given query. https://python.langchain.com/v0.1/docs/integrations/vectorstores/faiss/

*   LangChain equips us with advanced text generation techniques, enabling our query engine to generate meaningful and context-aware responses. https://python.langchain.com/v0.1/docs/get_started/introduction

### Dataset

*   The legal dataset provided includes the consolidated acts and regulations of Canada in both English and French as a collection of XML documents which are regularly updated in the linked repository. This dataset will serve as the source for the retrieval component of the RAG, enabling the LLaMA3 model to access and utilize specific legal information when generating responses. https://github.com/justicecanada/laws-lois-xml

# Data Download & Preprocessing

We will be utilizing FAISS as our vector store. To begin we must install the required prerequisite libraries and process and embed our XML document data into a FAISS index.

In [6]:
!pip install langchain;
!pip install langchain-core;
!pip install langchain-community;
!pip install langchain_experimental;
!pip install langchain-text-splitters
!pip install langchain-sentence-transformers
!pip install langchainhub
!pip install gpt4all
!pip install langchain-chroma
!pip install unstructured;

ERROR: Could not find a version that satisfies the requirement langchain-sentence-transformers (from versions: none)
ERROR: No matching distribution found for langchain-sentence-transformers


### Clone repository

In [7]:
from google.colab import drive
import os
google_drive_mnt = '/content/drive'
root_google_drive = os.path.join(google_drive_mnt, 'My Drive')
data_directory_name = 'Laws'
data_directory_path = os.path.join(root_google_drive, data_directory_name)
github_url = "https://github.com/justicecanada/laws-lois-xml/"
drive.mount(google_drive_mnt, force_remount=True)
!git clone "$github_url" "$data_directory_path" || (git -C "$data_directory_path" pull)

Mounted at /content/drive
fatal: destination path '/content/drive/My Drive/Laws' already exists and is not an empty directory.
Already up to date.


###Load xml docs into langchain documents

In [8]:
import os
from langchain_community.document_transformers import BeautifulSoupTransformer
from langchain_community.document_loaders import DirectoryLoader, UnstructuredXMLLoader
import nltk
nltk.download('averaged_perceptron_tagger')
path_to_acts = os.path.join(data_directory_path, 'eng/acts/')
loader = DirectoryLoader(path_to_acts, glob="**/A*.xml", loader_cls=UnstructuredXMLLoader, show_progress=True, use_multithreading=True, loader_kwargs={"mode":"elements"})
docs = loader.load()
print(len(docs))
print(docs[0])
##TODO: It appears a lot of the xml gets removed when loaded.  How do we keep it?  Convert to HTML using XLST and load the html?

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
100%|██████████| 36/36 [00:52<00:00,  1.45s/it]

21187
page_content='C-86' metadata={'source': '/content/drive/My Drive/Laws/eng/acts/A-1.3.xml', 'file_directory': '/content/drive/My Drive/Laws/eng/acts', 'filename': 'A-1.3.xml', 'last_modified': '2024-05-14T00:01:05', 'languages': ['eng'], 'filetype': 'application/xml', 'category': 'UncategorizedText'}


##Split langchain documents

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
split_docs = text_splitter.split_documents(docs)
for split_doc in split_docs:
  split_doc.metadata["languages"] = 'eng'
print(split_docs[0])
print(split_docs[1])
print(len(split_docs))
#TODO: No idea of RecursiveCharacterTextSplitter is the right splitter for xml data

page_content='C-86' metadata={'source': '/content/drive/My Drive/Laws/eng/acts/A-1.3.xml', 'file_directory': '/content/drive/My Drive/Laws/eng/acts', 'filename': 'A-1.3.xml', 'last_modified': '2024-05-14T00:01:05', 'languages': 'eng', 'filetype': 'application/xml', 'category': 'UncategorizedText'}
page_content='1' metadata={'source': '/content/drive/My Drive/Laws/eng/acts/A-1.3.xml', 'file_directory': '/content/drive/My Drive/Laws/eng/acts', 'filename': 'A-1.3.xml', 'last_modified': '2024-05-14T00:01:05', 'languages': 'eng', 'filetype': 'application/xml', 'category': 'UncategorizedText'}
21172


### Create embeddings for the split documents ~ approximately 10 minutes

In [10]:
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
vectorstore = Chroma.from_documents(documents=split_docs, embedding=GPT4AllEmbeddings())

### Retrieve documents related to question.
Modify the mode of transport in the question.  Note that results about railway transportation are returned if you use "trains" as the mode of transport.  This is the benefit of semantic search over straight text search.

In [11]:
question = "What statutes affect the train mode of transportation?"
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.2}
)
retrieved_docs = retriever.invoke(question)
print(retrieved_docs)
##TODO: Sometimes the page_content returned is just 'transportation.'

[Document(page_content='Railway Safety Act', metadata={'category': 'Title', 'file_directory': '/content/drive/My Drive/Laws/eng/acts', 'filename': 'A-1.xml', 'filetype': 'application/xml', 'languages': 'eng', 'last_modified': '2024-05-14T00:01:06', 'source': '/content/drive/My Drive/Laws/eng/acts/A-1.xml'}), Document(page_content='Railway Safety Act', metadata={'category': 'Title', 'file_directory': '/content/drive/My Drive/Laws/eng/acts', 'filename': 'A-1.xml', 'filetype': 'application/xml', 'languages': 'eng', 'last_modified': '2024-05-14T00:01:06', 'source': '/content/drive/My Drive/Laws/eng/acts/A-1.xml'}), Document(page_content='Canada Transportation Act', metadata={'category': 'Title', 'file_directory': '/content/drive/My Drive/Laws/eng/acts', 'filename': 'A-0.6.xml', 'filetype': 'application/xml', 'languages': 'eng', 'last_modified': '2024-05-14T00:01:05', 'source': '/content/drive/My Drive/Laws/eng/acts/A-0.6.xml'}), Document(page_content='Canada Transportation Act', metadata={

### RAG chain

In [15]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import GPT4All
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

model_filename = 'mistral-7b-openorca.gguf2.Q4_0.gguf'
model_url = f'https://gpt4all.io/models/gguf/{model_filename}'
model_directory_name = 'models'
model_path = os.path.join(root_google_drive, model_directory_name, model_filename)
!  [[ -e "$model_path" ]] || curl -L --silent -o "$model_path" "$model_url"
model = GPT4All(model=model_path, n_threads=8)
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)




Answer: The Canada Transportation Act is a significant piece of legislation that affects the transportation sector in Canada. It establishes the framework for the regulation, development, and management of various modes of transportation within the country, including railways, roadways, air transport, and waterborne shipping. By setting standards, providing guidelines, and ensuring fair competition among different carriers, this act plays a crucial role in maintaining an efficient and safe transportation system across Canada.


## Invoke RAG chain ~ 20 minutes

In [16]:
response = retrieval_chain.invoke({"input": "What statutes affect Canadian Transportation and how?"})
print(response["answer"])



Answer: The Canada Transportation Act is a significant piece of legislation that affects Canadian transportation. It establishes the framework for the regulation, development, and management of various modes of transportation in Canada, including railways, roads, air transport, and waterways. This act ensures that these different sectors work together efficiently while maintaining safety standards and promoting economic growth within the country.


# Setting Up the Environment

In [ ]:
# Use pip to install all dependencies required by the LangChain agents
# NOTE: For a local, stable environment, I would handle the installation of dependencies outside of notebook code
# For the purpose of this project, I'll outline the dependencies here, and test them in Google CoLab

if 'google.colab' in str(get_ipython()):
    !pip install openai;
    !pip install pandas;
    !pip install matplotlib;
    !pip install seaborn;
    !pip install cohere;
    !pip install tiktoken;
    !pip install pypdf;
    !pip install faiss-gpu;
    !pip install google-search-results;

    !pip install cuda-python
    !pip install huggingface_hub
    !pip install unstructured
    !pip install sentence-transformers
    !pip install numpy
    !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --upgrade --no-cache-dir --verbose
else:
    print("Executing locally")

In [ ]:
# This code detects the environment where we're running, and in the free version of Google Colab will distinguish between
# a T4 GPU, a TPU, or CPU execution. It will populate the environment and GPU status accordingly, and warn if we're executing on a CPU
# In my local environment, it will use the AMD hipinfo utility to identify the available GPU. Easily adapted to CUDA, for nvidia

if 'google.colab' in str(get_ipython()):
    environment = "Google Colab"
    gpu_id = !nvidia-smi -L
    if "command not found" in str(gpu_id):
        gpu_id = "WARNING: GPU not configured"
else:
    environment = "Local execution"
    hipinfo = !hipinfo
    elements = hipinfo.n.split('\n')
    for line in elements:
        if 'Name:' in line:
            gpu_id = line.split('Name:', 1)[-1].strip()
            break

In [ ]:
# Module versions, execution environment and GPU availability
print("Execution environment:",environment)
print("GPU Available:",gpu_id,"\n")
#print("OpenAI version:",openai.__version__,"\n")
#print("LangChain version:",langchain.__version__)
#print("Langchain Core version:",langchain_core.__version__)
#print("LangChain Experimental version:",langchain_experimental.__version__)

# Large Language Model (LLM) Setup

Downloading the Llama 3 8B GGUF model from NousResearch on Hugging Face

The model we will be downloading from Hugging Face is a **5-bit quantized version of the Llama 3 8B chat model**, made available by NousResearch. The model is made available in the **GGUF format** - a new format introduced by the Llama CPP team and a replacement for the earlier GGML format, with advantages such as better tokenization and support for special tokens.

In [ ]:
from huggingface_hub import hf_hub_download

In [ ]:
model_name_or_path = "NousResearch/Meta-Llama-3-8B-Instruct-GGUF"
model_basename = "Meta-Llama-3-8B-Instruct-Q5_K_M.gguf" # the model is in gguf format

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

---

## Load Dataset XML files

Use langchain xml loader which is a wrapper for unstructured library: https://unstructured-io.github.io/unstructured/core.html.

Unstructured supports chunking but unsure how to access that functionality through the langain UnstructureXMLLoader wrapper.

The loader.load() call returns an array of langchain Documents. https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document

In [ ]:
# I borrowed this code from another project in an attempt to understand how to parse all of our XML files into the vector store.
# We will need to swap the JSON loader for an XML loader at the very least.
# Add your code for downloading the XML files here @Matt

import functools
import pathlib
from typing import Any

import langchain.text_splitter
import langchain_community.document_loaders
import langchain_community.embeddings
import langchain_community.vectorstores.faiss
import langchain_core.documents
import sentence_transformers
import torch

# A custom configuration of the SentenceTransformer class is defined using functools.partial, specifying the transformer should run on the CPU and trust remote code.
SentenceTransformer = functools.partial(sentence_transformers.SentenceTransformer, trust_remote_code=True, device="cpu")

# This function is intended to update metadata for a record by extracting and modifying metadata information based on the record's contents:
# - It adjusts the topic by extracting the parent directory name from the source if available.
# - Updates the source and title from the record data.

def metadata_func(record: dict[str, Any], metadata: dict[str, Any]) -> dict[str, Any]:
    """Extract metadata from a record.

    Args:
        record: The record.
        metadata: The default metadata generated by the JSONLoader.

    Returns:
        The updated metadata.
    """
    if "source" in metadata:
        metadata["topic"] = pathlib.Path(metadata["source"]).parent.name
    metadata["source"] = record.get("url")
    metadata["title"] = record.get("title")

    return metadata

# This function deduplicates document chunks:
# - It takes a list of document chunks and filters out duplicates based on the content of the chunks (page_content), ensuring that only unique chunks are retained.

def dedup_chunks(
    chunks: list[langchain_core.documents.Document],
) -> list[langchain_core.documents.Document]:
    """Deduplicate chunks based on their page content.

    Args:
        chunks: A list of chunks.

    Returns:
        A list of deduplicated chunks.
    """
    deduped_chunks = []

    chunk_set = set()
    for chunk in chunks:
        if chunk.page_content not in chunk_set:
            chunk_set.add(chunk.page_content)
            deduped_chunks.append(chunk)
    return deduped_chunks

# The core function of the script, which orchestrates the creation of a FAISS database:
# - Loading Documents: Utilizes DirectoryLoader to load documents from a specified directory. The documents are expected to be in JSON format.
# - Splitting Documents: Documents are split into smaller chunks using a SentenceTransformersTokenTextSplitter, which leverages a specified embedding model.
# - Deduplication: The chunks are deduplicated using the dedup_chunks function.
# - Embedding Documents: Document chunks are embedded using the specified transformer model. Optionally, the model can be set to use half precision.
# - Creating FAISS Index: A FAISS index is created from the deduplicated and embedded chunks, which is then saved locally.

def create_db(
    data_path: str = "/data_fast/laws-lois-xml/documents",
    embedding_model: str = "NousResearch/Meta-Llama-3-8B-Instruct-GGUF",
    save_path: str = "/data_fast/laws-lois-xml/Meta-Llama-3-8B-Instruct-GGUF/faiss",
    half_precision: bool = False,
) -> langchain_community.vectorstores.faiss.FAISS:
    """Create a faiss db from a directory of JSON files.

    Args:
        data_path: Path to the directory of JSON files.
        embedding_model: The HuggingFace model name to use for embeddings.
        save_path: Path to save the db.
        half_precision: Whether to use half precision for the embedding model.

    Returns:
        A faiss db.
    """
    loader = langchain_community.document_loaders.DirectoryLoader(
        data_path,
        glob="**/*.json",
        loader_cls=langchain_community.document_loaders.JSONLoader,  # pyright: ignore[reportArgumentType]
        loader_kwargs={
            "jq_schema": ".",
            "metadata_func": metadata_func,
            "content_key": "article",
        },
        use_multithreading=True,
        recursive=True,
    )
    docs = loader.load()
    with mock.patch.object(sentence_transformers, "SentenceTransformer", new=SentenceTransformer):
        splitter = langchain.text_splitter.SentenceTransformersTokenTextSplitter(model_name=embedding_model)
    chunks = splitter.split_documents(docs)
    deduped_chunks = dedup_chunks(chunks)

    embedder = langchain_community.embeddings.HuggingFaceEmbeddings(
        model_name=embedding_model,
        show_progress=True,
        model_kwargs={"trust_remote_code": True},
    )
    assert isinstance(embedder.client, torch.nn.Module)
    if half_precision:
        embedder.client.half()
    db = langchain_community.vectorstores.faiss.FAISS.from_documents(deduped_chunks, embedder)
    db.save_local(save_path)
    return db



# Retrieval-Augmented Generation

#Takeaways
While the Llama 3 LangChain agent is definitely capable of providing answers and also using the external RAG vector store to compute the right answer to the prompt, the LangChain ReAct Prompt Template is very specific and it seems the LLM's chain does not stop even after it arrives at the correct answer.

This is an issue with working with open-source LLMs in combination with LangChain on the free tier of Google Colab - due to the Colab GPU's 13 GB memory limit on the free tier, we are restricted to working with the 8B model of Llama 3, which is not as good at following instructions as OpenAI's GPT models.